In [6]:
# prompt: code to convert csv file to txt file and remove all the extra \t and \n in that file

import csv
import re

def convert_csv_to_txt(csv_filepath, txt_filepath):
    """Converts a CSV file to a TXT file, removing extra tabs and newlines."""
    try:
        with open(csv_filepath, 'r', encoding='utf-8') as csvfile, open(txt_filepath, 'w', encoding='utf-8') as txtfile:
            reader = csv.reader(csvfile)
            for row in reader:
                cleaned_row = [re.sub(r'[\t\n]+', '', item) for item in row]  # Remove tabs and newlines from each item
                txtfile.write(','.join(cleaned_row) + '\n')  # Join items back with commas and write to txt file
        print(f"Successfully converted '{csv_filepath}' to '{txt_filepath}'")

    except FileNotFoundError:
        print(f"Error: File '{csv_filepath}' not found.")
    except Exception as e:
        print(f"An error occurred: {e}")

# Example usage: Replace 'input.csv' and 'output.txt' with your filepaths.
convert_csv_to_txt('PoetryFoundationData.csv', 'output.txt')

Successfully converted 'PoetryFoundationData.csv' to 'output.txt'


In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import tensorflow.keras.utils as ku 
from wordcloud import WordCloud 
from tensorflow.keras.preprocessing.sequence import pad_sequences 
from tensorflow.keras.layers import Embedding, LSTM, Dense, Dropout, Bidirectional 
from tensorflow.keras.preprocessing.text import Tokenizer 
from tensorflow.keras.models import Sequential 
from tensorflow.keras.optimizers import Adam 
from tensorflow.keras import regularizers

# Reading the text data file 
data = open('output.txt', encoding="utf8").read() 

# EDA: Generating WordCloud to visualize 
# the text 
wordcloud = WordCloud(max_font_size=50, 
					max_words=100, 
					background_color="black").generate(data) 

# Plotting the WordCloud 
plt.figure(figsize=(8, 4)) 
plt.imshow(wordcloud, interpolation='bilinear') 
plt.axis("off") 
plt.savefig("WordCloud.png") 
plt.show() 


# Generating the corpus by 
# splitting the text into lines 
corpus = data.lower().split("\n") 
print(corpus[:10]) 


# Fitting the Tokenizer on the Corpus 
tokenizer = Tokenizer() 
tokenizer.fit_on_texts(corpus) 

# Vocabulary count of the corpus 
total_words = len(tokenizer.word_index) 

print("Total Words:", total_words) 


# Converting the text into embeddings 
input_sequences = [] 
for line in corpus: 
	token_list = tokenizer.texts_to_sequences([line])[0] 

	for i in range(1, len(token_list)): 
		n_gram_sequence = token_list[:i+1] 
		input_sequences.append(n_gram_sequence) 

max_sequence_len = max([len(x) for x in input_sequences]) 
input_sequences = np.array(pad_sequences(input_sequences, 
										maxlen=max_sequence_len, 
										padding='pre')) 
predictors, label = input_sequences[:, :-1], input_sequences[:, -1] 
label = ku.to_categorical(label, num_classes=total_words+1) 


# Building a Bi-Directional LSTM Model 
model = Sequential() 
model.add(Embedding(total_words+1, 100, 
					input_length=max_sequence_len-1)) 
model.add(Bidirectional(LSTM(150, return_sequences=True))) 
model.add(Dropout(0.2)) 
model.add(LSTM(100)) 
model.add(Dense(total_words+1/2, activation='relu', 
				kernel_regularizer=regularizers.l2(0.01))) 
model.add(Dense(total_words+1, activation='softmax')) 
model.compile(loss='categorical_crossentropy', 
			optimizer='adam', metrics=['accuracy']) 
print(model.summary()) 


history = model.fit(predictors, label, epochs=150, verbose=1)


seed_text = "a short phrase"
next_words = 25
ouptut_text = "" 

for _ in range(next_words): 
	token_list = tokenizer.texts_to_sequences([seed_text])[0] 
	token_list = pad_sequences( 
		[token_list], maxlen=max_sequence_len-1, 
	padding='pre') 
	predicted = np.argmax(model.predict(token_list, 
										verbose=0), axis=-1) 
	output_word = "" 
	
	for word, index in tokenizer.word_index.items(): 
		if index == predicted: 
			output_word = word 
			break
			
	seed_text += " " + output_word 
	
print(seed_text)

ModuleNotFoundError: No module named 'tensorflow'